<a href="https://colab.research.google.com/github/thebishorup/nlp/blob/master/FastText_NLP_Mulitilevel_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 130kB/s 
     |████████████████████████████████| 983kB 34.6MB/s 
     |████████████████████████████████| 245kB 37.9MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
     |████████████████████████████████| 471kB 38.6MB/s 
     |████████████████████████████████| 1.1MB 39.4MB/s 
     |████████████████████████████████| 3.0MB 30.4MB/s 
     |████████████████████████████████| 890kB 37.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.21.3-cp36-none-any.whl size=25270203 sha256=d9c9a790c49070d0dbd9b18f6c4c303dbd0248f874cc4d605ecd829a3c28d813
  Stored in directory: /root/.cache/pip/wheels/d6/ef/6d/9428365b27ac1e84a77214c5718e7f8b2180007af593791247
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=3f44c5530af9baa95e399931dad492e0092d814ec948998d1280d9cbd57f46bb
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
!git clone https://github.com/laxmimerit/Toxic-Comment.git

Cloning into 'Toxic-Comment'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [3]:
import pandas as pd
import numpy as np


In [5]:
df = pd.read_csv('/content/Toxic-Comment/train.csv')

In [6]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
df.shape

(159571, 8)

In [12]:
import ktrain
from ktrain import text

In [13]:
NUM_WORDS = 50000
MAXLEN = 150


In [20]:
train, val, preproc = text.texts_from_csv('/content/Toxic-Comment/train.csv', 'comment_text', ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], 
                                          ngram_range=1, max_features=NUM_WORDS, maxlen=MAXLEN)

detected encoding: utf-8 (if wrong, set manually)
language: en
Word Counts: 196935
Nrows: 143613
143613 train sequences
train sequence lengths:
	mean : 67
	95percentile : 228
	99percentile : 565
x_train shape: (143613,150)
y_train shape: (143613, 6)
Is Multi-Label? True
15958 test sequences
test sequence lengths:
	mean : 66
	95percentile : 227
	99percentile : 580
x_test shape: (15958,150)
y_test shape: (15958, 6)


In [21]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [22]:
model = text.text_classifier('fasttext', train_data=train, preproc=preproc)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
done.


In [23]:
learner = ktrain.get_learner(model, train, val)

In [24]:
learner.autofit(lr = 0.001, epochs=2)



begin training using triangular learning rate policy with max lr of 0.001...
Epoch 1/2
4488/4488 [==============================] - 179s 40ms/step - loss: 0.1435 - accuracy: 0.7185 - val_loss: 0.0548 - val_accuracy: 0.9939
Epoch 2/2
4488/4488 [==============================] - 170s 38ms/step - loss: 0.0635 - accuracy: 0.9913 - val_loss: 0.0506 - val_accuracy: 0.9939


In [25]:
predictor = ktrain.get_predictor(learner.model, preproc=preproc)

In [26]:
predictor.predict(['This is such a amazing course.'])

[[('toxic', 0.012342818),
  ('severe_toxic', 2.6129945e-07),
  ('obscene', 0.00049738464),
  ('threat', 1.6112053e-05),
  ('insult', 0.001189647),
  ('identity_hate', 0.00010970853)]]

In [27]:
predictor.predict(['You are such a stupid person I have ever met'])

[[('toxic', 0.92669266),
  ('severe_toxic', 0.15384819),
  ('obscene', 0.6560456),
  ('threat', 0.03859564),
  ('insult', 0.6112126),
  ('identity_hate', 0.1390605)]]

In [28]:
# save the model
predictor.save('/content/drive/My Drive/NLP_TRAINED_MODEL/toxic_fasttext')